Imports and input

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output, State
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import h3
import json

file = 'data/traffic.parquet'


df = pd.read_parquet(file, engine='fastparquet')
df = df.sample(1000000)

Data transformation

In [4]:
df["Start_Time"] = pd.to_datetime(df["Start_Time"])
df["hour"] = df["Start_Time"].dt.hour
df["weekday"] = df["Start_Time"].dt.day_name()
df["month"] = df["Start_Time"].dt.month
df["year"] = df["Start_Time"].dt.year

# Seasons
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"
df["season"] = df["month"].apply(get_season)


standalone hourly/monthly/seasonal vis

In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"
df["season"] = df["month"].apply(get_season)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Accidents Over Time"),
    dcc.RadioItems(
        id="time-granularity",
        options=[
            {"label": "Hourly", "value": "hour"},
            {"label": "Weekly", "value": "weekday"},
            {"label": "Monthly", "value": "month"},
            {"label": "Seasonal", "value": "season"},
        ],
        value="hour",   # default
        inline=True
    ),
    dcc.Graph(id="time-chart")
])

@app.callback(
    Output("time-chart", "figure"),
    Input("time-granularity", "value")
)
def update_chart(granularity):
    counts = df.groupby(granularity).size().reset_index(name="count")

    # nice ordering
    if granularity == "weekday":
        order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
        counts[granularity] = pd.Categorical(counts[granularity], categories=order, ordered=True)
        counts = counts.sort_values(granularity)
    elif granularity == "month":
        month_names = {
            1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun",
            7:"Jul", 8:"Aug", 9:"Sep", 10:"Oct", 11:"Nov", 12:"Dec"
        }
        counts["month"] = counts["month"].map(month_names)

    # histogram for all cases
    fig = px.bar(
        counts,
        x=granularity,
        y="count",
        title=f"Accidents by {granularity.capitalize()}",
        text="count"
    )

    return fig

if __name__ == "__main__":
    app.run(debug=True)


standalone avg precip by severity

In [ ]:
fig_precip = px.bar(
    df.groupby("Severity", as_index=False)["Precipitation(in)"].mean(),
    x="Severity", y="Precipitation(in)",
    title="Average Precipitation by Severity"
)
fig_precip.show()

Main layout

In [ ]:
# --- Initialize the Dash App with Bootstrap and FontAwesome for icons ---
# Dash will automatically serve any CSS file placed in an 'assets' folder.
app = dash.Dash(
    __name__,
    external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME]
)

# --- App Layout ---
app.layout = html.Div([
    # Main container for the entire layout grid
    dbc.Row(
        [
            # --- Main Content Column (Map + Charts) ---
            dbc.Col(
                html.Div(
                    [
                        # Map Container
                        html.Div(
                            html.Div("MAP PLACEHOLDER", className="placeholder"),
                            id="map-container",
                            className="map-container"
                        ),

                        # Charts Container (initially hidden)
                        html.Div(
                            dbc.Row(
                                [
                                    # --- CHART 1 GOES HERE (Now with Inline Controls) ---
                                    dbc.Col(
                                        html.Div([
                                            # Container for inline Title and Dropdown
                                            html.Div([
                                                html.H4("Severity by", className="me-2 mb-0"),
                                                dcc.Dropdown(
                                                    id='chart-1-dropdown',
                                                    options=[
                                                        {'label': 'Precipitation', 'value': 'Precipitation(in)'},
                                                        {'label': 'Temperature', 'value': 'Temperature(F)'}
                                                    ],
                                                    value='Precipitation(in)', # Default value
                                                    clearable=False,
                                                    style={'width': '200px'}
                                                )
                                            ], className="d-flex justify-content-center align-items-center mb-3"),

                                            dcc.Graph(
                                                id='chart-1',
                                                figure={},
                                                config={'responsive': True},
                                                className="flex-grow-1"
                                            )
                                        ], className="d-flex flex-column h-100"),
                                        width=6,
                                        className="p-3",
                                        style={'height': '100%'}
                                    ),
                                    # --- CHART 2 GOES HERE (with Flexbox fix) ---
                                    dbc.Col(
                                        # Use flexbox to manage vertical space. This container
                                        # will be a vertical flexbox that fills the height.
                                        html.Div([
                                            html.H4("Accidents Over Time", className="text-center"),
                                            dcc.RadioItems(
                                                id="time-granularity",
                                                options=[
                                                    {"label": "Hourly", "value": "hour"},
                                                    {"label": "Weekly", "value": "weekday"},
                                                    {"label": "Monthly", "value": "month"},
                                                    {"label": "Seasonal", "value": "season"},
                                                ],
                                                value="hour",
                                                inline=True,
                                                # Add gap-4 for spacing between radio buttons
                                                className="d-flex justify-content-center mb-3 gap-4"
                                            ),
                                            # This Graph will grow to fill the remaining space.
                                            # Initialize with an empty figure to prevent cut-off on first load.
                                            dcc.Graph(
                                                id="time-chart",
                                                figure={}, # Fix for initial load cut-off
                                                config={'responsive': True},
                                                className="flex-grow-1" # Bootstrap class for flex-grow: 1
                                            )
                                        ], className="d-flex flex-column h-100"), # Flex column, height 100%
                                        width=6,
                                        className="p-3",
                                        style={'height': '100%'}
                                    ),
                                ],
                                className="h-100 g-0"
                            ),
                            id="charts-container",
                            className="charts-container"
                        ),
                    ],
                    className="main-content-wrapper"
                ),
                id="main-content-col",
                width=12
            ),

            # --- Filter Sidebar Column (initially hidden) ---
            dbc.Col(
                html.Div("FILTERS PLACEHOLDER", className="placeholder", style={"flex-direction": "column", "gap": "20px"}),
                id="filter-sidebar-col",
                width=0
            ),
        ],
        className="g-0",
        style={"height": "100vh"}
    ),

    # --- Toggle Buttons ---
    html.Button(
        [html.I(className="fas fa-chevron-up me-2"), "Show Charts"],
        id="charts-toggle-button",
        className="toggle-button charts-toggle",
        n_clicks=0
    ),
    html.Button(
        [html.I(className="fas fa-chevron-left me-2"), "Show Filters"],
        id="filters-toggle-button",
        className="toggle-button filters-toggle",
        n_clicks=0
    ),
])


# --- Callbacks for Interactivity ---

# Callback to toggle the visibility of the bottom charts panel
@app.callback(
    [Output("map-container", "style"),
     Output("charts-container", "style"),
     Output("charts-toggle-button", "children")],
    [Input("charts-toggle-button", "n_clicks")]
)
def toggle_charts_panel(n_clicks):
    if n_clicks is not None and n_clicks % 2 == 1:
        # State when charts are VISIBLE
        map_style = {'height': '50vh'}
        charts_style = {'height': '50vh', 'visibility': 'visible'}
        button_content = [html.I(className="fas fa-chevron-down me-2"), "Hide Charts"]
    else:
        # State when charts are HIDDEN
        map_style = {'height': 'calc(100vh - 50px)'}
        # Use visibility: hidden and height: 0 instead of display: none
        charts_style = {'height': '0', 'visibility': 'hidden'}
        button_content = [html.I(className="fas fa-chevron-up me-2"), "Show Charts"]
    return map_style, charts_style, button_content


# Callback to toggle the visibility of the right filter sidebar
@app.callback(
    [Output("main-content-col", "width"),
     Output("filter-sidebar-col", "width"),
     Output("filter-sidebar-col", "className"),
     Output("filters-toggle-button", "children")],
    [Input("filters-toggle-button", "n_clicks")]
)
def toggle_filters_sidebar(n_clicks):
    if n_clicks is not None and n_clicks % 2 == 1:
        main_col_width = 10
        filter_col_width = 2
        filter_className = "p-3"
        button_content = [html.I(className="fas fa-chevron-right me-2"), "Hide Filters"]
    else:
        main_col_width = 12
        filter_col_width = 0
        filter_className = ""
        button_content = [html.I(className="fas fa-chevron-left me-2"), "Show Filters"]
    return main_col_width, filter_col_width, filter_className, button_content

# --- Callback for Chart 1 ---
@app.callback(
    Output('chart-1', 'figure'),
    Input('chart-1-dropdown', 'value')
)
def update_chart_1(selected_metric):
    title_text = selected_metric.split('(')[0] # "Precipitation" or "Temperature"

    if selected_metric == 'Precipitation(in)':
        # Create Bar Chart
        agg_df = df.groupby("Severity", as_index=False)[selected_metric].mean()
        fig = px.bar(
            agg_df,
            x="Severity",
            y=selected_metric
        )
    else: # 'Temperature(F)'
        # Create Box Plot
        fig = px.box(
            df,
            x="Severity",
            y=selected_metric
        )

    fig.update_layout(margin=dict(t=10, b=10, l=10, r=10), yaxis_title=title_text)
    return fig

# --- Callback for Chart 2 ---
@app.callback(
    Output("time-chart", "figure"),
    Input("time-granularity", "value")
)
def update_chart(granularity):
    counts = df.groupby(granularity).size().reset_index(name="count")

    if granularity == "weekday":    
        order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        counts[granularity] = pd.Categorical(counts[granularity], categories=order, ordered=True)
        counts = counts.sort_values(granularity)
    elif granularity == "month":
        month_names = {
            1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun",
            7:"Jul", 8:"Aug", 9:"Sep", 10:"Oct", 11:"Nov", 12:"Dec"
        }
        month_order = list(month_names.values())
        df_months = pd.DataFrame({'month_num': month_names.keys(), 'month_name': month_names.values()})
        counts = pd.merge(counts, df_months, left_on='month', right_on='month_num')
        counts['month_name'] = pd.Categorical(counts['month_name'], categories=month_order, ordered=True)
        counts = counts.sort_values('month_name')
        granularity = 'month_name'


    fig = px.bar(
        counts,
        x=granularity,
        y="count",
        text_auto=True
    )
    # Update layout to remove Y-axis title and capitalize X-axis title
    fig.update_layout(
        margin=dict(t=10, b=10, l=10, r=10),
        yaxis_title=None, # Remove y-axis title ("count")
        xaxis_title=granularity.replace('_name', '').capitalize() # Capitalize x-axis title
    )
    return fig

# --- Run the App ---
if __name__ == '__main__':
    app.run(debug=True)



[2025-10-28 21:12:51,064] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/pjo/uni/sem8/datavis/davi_traff_weath/.venv/lib/python3.12/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pjo/uni/sem8/datavis/davi_traff_weath/.venv/lib/python3.12/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pjo/uni/sem8/datavis/davi_traff_weath/.venv/lib/python3.12/site-packages/dash/dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
                    ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pjo/uni/sem8/datavis/davi_traff_weath/.venv/lib/python3.12/site-packages/dash/_callback.py", line 688, in add_context
    raise err
  